# Chapter: 14
## Section: RLHF with PyTorch

In [ ]:
!pip install torch
!pip install transformers

In [ ]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from torch import optim
from torch.utils.data import DataLoader

# Pretrain a GPT-2 language model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')
optimizer = optim.Adam(model.parameters(), lr=1e-3)

This step forward does not run before you determine the dataset object, assumed to be data colleced by human for RLHF process.

In [ ]:
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

In [ ]:
class Reward_Model(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RewardModel, self).__init__()
        self.fc_layer1 = torch.nn.Linear(input_size, hidden_size)
        self.fc_layer2 = torch.nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = torch.relu(self.fc_layer1(x))
        x = self.fc_layer2(x)
        return x


In [ ]:
reward_model = Reward_Model(input_size, hidden_size, output_size)

In [ ]:
for epoch in range(n_epochs):
    for batch in dataloader:
        input_ids = tokenizer.encode(batch['input'], return_tensors='pt')
        output_ids = tokenizer.encode(batch['output'], return_tensors='pt')
        reward = reward_model(batch['input'])
        loss = model(input_ids, labels=output_ids).loss * reward
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()